In [13]:
from typing import Tuple, Optional, List, Union

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import torch.nn.functional as F

In [217]:
def fit_model(model: nn.Module, input_data: torch.Tensor=None, labels: torch.Tensor=None, loader=None, epoches: int = 10,
              lr=0.001, batch_size: int = 32, criterion=None):
    criterion = criterion or nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    if not loader:
        dataset = TensorDataset(input_data, labels)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(epoches):
        for data, target in loader:
            optimizer.zero_grad()
            output = model(data.float()).squeeze()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    return model


def fit_model_with_states(model: nn.Module, input_data: torch.Tensor=None, labels: torch.Tensor=None, loader=None, epoches: int = 10,
                          lr=0.001, batch_size: int = 32):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    if not loader:
        dataset = TensorDataset(input_data, labels)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(epoches):
        model.train()
        total_loss = 0
        # hidden = model.init_hidden(batch_size)
        for data, target in loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.size(0)
        print(f'Epoch {epoch + 1}, Average Loss: {total_loss / len(loader.dataset)} [{loss.item()}]')
    model.eval()
    return model


def load_model(model: nn.Module, filepath: str):
    model.load_state_dict(torch.load(filepath))
    return model


def list_to_tensor(lst: List, dims: int = 2) -> torch.Tensor:
    tensor = torch.tensor(lst, dtype=torch.float32)
    if dims == 3:
        tensor = tensor.unsqueeze(-1)
    return tensor.to('cuda')


def dataframe_to_tensor(df: pd.DataFrame, dims: int = 2) -> torch.Tensor:
    return list_to_tensor(df.values, dims)


def tensor_to_list(tensor: torch.Tensor) -> list:
    return tensor.detach().cpu().tolist()

In [15]:
class AttentionModel(nn.Module):
    def __init__(self, input_size: int, hidden_sizes: Optional[List[int]] = None,
                 output_size: int = 1):
        super(AttentionModel, self).__init__()
        hidden_sizes = hidden_sizes or [30]
        # Веса внимания для входных фич
        self.attention_weights = nn.Parameter(torch.randn(input_size, requires_grad=True))

        self.layer1 = nn.Linear(input_size, hidden_sizes[0])
        self.layer2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.layer3 = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [245]:
class PredictNextNumModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int = 16, output_size: int = 1):
        super(PredictNextNumModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.rnn2 = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.rnn3 = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        hidden = self.init_hidden(x.size(0))
        out, hidden = self.rnn(x, hidden)
        out, hidden = self.rnn2(out, hidden)
        out, hidden = self.rnn3(out, hidden)
        out = self.linear(out)
        return out[:, -1, :]

    def init_hidden(self, batch_size: int):
        return torch.zeros(1, batch_size, self.hidden_size).to('cuda')

In [210]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Сверточный слой #1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # Принимает 1 канал, выдает 16 каналов
        # Сверточный слой #2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # Принимает 16 каналов, выдает 32 канала
        # Пуллинг слой
        self.pool = nn.MaxPool2d(2, 2)  # Пуллинг с окном 2x2
        # Линейный слой для классификации
        self.fc = nn.Linear(32 * 7 * 7, 10)  # предполагаем, что изображение уменьшилось до 7x7

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Применяем ReLU и пуллинг после первого сверточного слоя
        x = self.pool(F.relu(self.conv2(x)))  # Применяем ReLU и пуллинг после второго сверточного слоя
        x = torch.flatten(x, 1)  # Выравнивание всех признаков в один вектор
        x = self.fc(x)  # Пропускаем через полносвязный слой
        return x

In [188]:
def next_number_func(start: float, count: int = 1) -> Union[List, float]:
    ans = [start]
    for i in range(count):
        x = ans[-1] * 2 + 1
        ans.append(x)
    ans.pop(0)
    return ans if len(ans) != 1 else ans[0]

In [222]:
input_size = 4
columns = [f'x{i + 1}' for i in range(input_size)]
# train_data = pd.DataFrame(np.random.randint(1, 100, size=(10000, input_size)), columns=columns)
train_data = pd.DataFrame([next_number_func(np.random.randint(1, 100), input_size) for _ in range(1000)],
                          columns=columns)

# target_data = pd.DataFrame([next_number_func(start=i) for i in train_data[f'x{len(columns)}'].to_list()],
#                            columns=['target'])
target_data = pd.DataFrame(train_data[f'x{len(columns)}'].to_list(), columns=['target'])

input_data = dataframe_to_tensor(train_data, dims=3)
labels = dataframe_to_tensor(target_data)

filename = 'predict_next'

In [111]:
# Загрузка модели из файла
model = load_model(PredictNextNumModel(1, 64, 1), f'{filename}.pth').to('cuda')

In [246]:
# Обучение модели
model = PredictNextNumModel(1, 512, 1)
model = model.to('cuda')

In [ ]:
fit_model_with_states(model, input_data, labels, epoches=100, batch_size=32)
torch.save(model.state_dict(), f'{filename}.pth')  # Сохранение весов модели

In [235]:
test_data = pd.DataFrame([[1, 3, 7, 15]])
model(dataframe_to_tensor(test_data, dims=3))

tensor([[5.7069]], device='cuda:0', grad_fn=<SliceBackward0>)

In [215]:
from torchvision import datasets, transforms

# Трансформации для предварительной обработки данных
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Загрузка данных
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [216]:
model = SimpleCNN()
model = fit_model(model, loader=train_loader, epoches=50, criterion=nn.CrossEntropyLoss())

Epoch 1, Loss: 0.005777818616479635
Epoch 2, Loss: 0.04960726201534271
Epoch 3, Loss: 0.006789261940866709
Epoch 4, Loss: 0.021904947236180305
Epoch 5, Loss: 0.024383459240198135
Epoch 6, Loss: 0.02011103369295597
Epoch 7, Loss: 0.03838662430644035
Epoch 8, Loss: 0.0060940515249967575
Epoch 9, Loss: 0.0004249002377036959
Epoch 10, Loss: 0.0015589382965117693
Epoch 11, Loss: 0.012817710638046265
Epoch 12, Loss: 0.005018122959882021
Epoch 13, Loss: 0.0008570727659389377
Epoch 14, Loss: 0.0025187439750880003
Epoch 15, Loss: 5.128941120347008e-05
Epoch 16, Loss: 0.0007193447090685368
Epoch 17, Loss: 0.003530555171892047
Epoch 18, Loss: 0.017555193975567818
Epoch 19, Loss: 3.407353869988583e-05
Epoch 20, Loss: 0.00303028360940516
Epoch 21, Loss: 1.5540445019723848e-05
Epoch 22, Loss: 9.602396858099382e-06
Epoch 23, Loss: 0.1825202852487564
Epoch 24, Loss: 0.039949383586645126
Epoch 25, Loss: 7.417173765134066e-05
Epoch 26, Loss: 1.551418063172605e-05
Epoch 27, Loss: 1.909787351905834e-05
Ep

In [ ]:
model(x)